# Data Preprocessing Tools

## Importing the libraries

In [ ]:
import numpy as np #to process arrays as inputs. "as np" to make a shortcut call in case we want to use numpy
import matplotlib.pyplot as plt #to plot charts. "." to use specific module pyplot and "plt" as shortcut
import pandas as pd #import data set and matrices. "pd" -> shortcut

## Importing the dataset

In [ ]:
dataset = pd.read_csv('Data.csv') #read data from csv file. It's a function from pandas
#Every dataset have features and dependant variables. Features -> Columns. Dependant variable -> what you want to predict (last column usually)
#NOTA -> caso os dados estejam separados por ";" --> dataset = pd.read_csv('winequality-red.csv', delimiter=';')


#Matrice (with features and respective values) EX: Country, age, salary
X = dataset.iloc[:, :-1].values #locate indexes (iloc) - vai procurar os indices rows,col , retira e coloca em X
#: -> takes everything in the range. (All the rows)
#:-1 -> takes every column except the last one (which is the dependant variable vector)

#Dependant Variable Vector (what you must predict) EX: Purchased (Yes or No)
Y = dataset.iloc[:, -1].values #will get all the rows of the last column

#Outro exemplo para serparar dados de variaveis dependentes e variaveis independentes
#
#X = dataset.drop('quality', axis=1)
#y = dataset['quality']
#
#quality é a

In [ ]:
print(X)

[['France' 44.0 72000.0]
 ['Spain' 27.0 48000.0]
 ['Germany' 30.0 54000.0]
 ['Spain' 38.0 61000.0]
 ['Germany' 40.0 nan]
 ['France' 35.0 58000.0]
 ['Spain' nan 52000.0]
 ['France' 48.0 79000.0]
 ['Germany' 50.0 83000.0]
 ['France' 37.0 67000.0]]


In [ ]:
print(Y)

['No' 'Yes' 'No' 'No' 'Yes' 'Yes' 'No' 'Yes' 'No' 'Yes']


## Taking care of missing data

In [ ]:
#We don't want missing data. So we must handle them
#1. Ignore observation by deleting the whole row - not a problem if you have much data
#2. Replace missing data by the AVG of all the values in the column.
#sometimes people also do Median, Modal, etc. But AVG is the most used

from sklearn.impute import SimpleImputer #library para data scientists. mt boa
imputer = SimpleImputer(missing_values=np.nan, strategy='mean') #create a new object of the SimpleImputer class. Não precisa de inputs
#deals missing values (np.nan) with the strategy mean (AVG)
#This just identifies the missing values, it doesn't actually replace them. For that - we do the fit method:
imputer.fit(X[:, 1:3]) #!!!only the numeric values!!!. no strings or booleans. We will only specify age and salary (numbers!) country no good.
#why 1:3? because we want the columns 1 and 2 (columns start counting in 0 and the upper bound isn't used by the function). Age and salary are
#1 and 2 so we will do 1:3
X[:, 1:3] = imputer.transform(X[:, 1:3]) #para transformar efetivamente. Como tem return das colunas transformadas, precisamos
#de as por no X novamente --> X[:, 1:3]

#caso quisesse ver dados nulos -> # Identify missing data    --> print(dataset.isnull().sum())


In [ ]:
print(X)

[['France' 44.0 72000.0]
 ['Spain' 27.0 48000.0]
 ['Germany' 30.0 54000.0]
 ['Spain' 38.0 61000.0]
 ['Germany' 40.0 63777.77777777778]
 ['France' 35.0 58000.0]
 ['Spain' 38.77777777777778 52000.0]
 ['France' 48.0 79000.0]
 ['Germany' 50.0 83000.0]
 ['France' 37.0 67000.0]]


## Encoding categorical data

In [ ]:
#Transformar features categóricas em números.
#Exemplo - France, Spain, Germany? da Coluna Country. Ordem NAO INTERESSA, LOGO APLICAR ONE HOT ENCODING.
#1. One-Hot encoding -> Transformar a coluna Country em três diferentes (porque existem três países diferentes)
#Cria vetores binários para cada um desses paises. EX: France (1,0,0), Spain (0,1,0) e Germany (0,0,1).
#Assim nao ha

#MT IMP - Um nome não é categorical data. Porquê? POrque é único. Após aplicar-se o One-Hot Encoding, iria fazer arrays enormes.
#Alem de tambem nao servir para qualquer tipo de predição...

### Encoding the Independent Variable

In [ ]:
#PARA APLICACOES -> BASTA ALTERAR A COLUNA REFERIDA NO OBJETO DE COLUMN TRANSFER (O 0) PELA COLUNA QUE QUEREMOS

#Country é uma variavel independente. Vamos mexer nisso
#Duas classes
from sklearn.compose import ColumnTransformer #transformar colunas
from sklearn.preprocessing import OneHotEncoder #para a tecnica one hot encoder

#Criar objetos
ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(), [0])], remainder='passthrough')
#queremos transformar com o onehotencoder na coluna 0 (Country). Remainder sao as restantes. Queremos manter portanto
#fica passthrough
#Solucao alternativa - caso quisesse escrever especificamente as colunas em vez dos seus indices:
#categorical_features = ['Sex', 'Embarked', 'Pclass']
# ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(), categorical_features)], remainder='passthrough')

#aplicar o fit e transformar o array das variaveis independentes
X = np.array(ct.fit_transform(X)) #np.array para obrigarmos a que o output da transformacao seja um array


In [ ]:
print(X)

[[1.0 0.0 0.0 44.0 72000.0]
 [0.0 0.0 1.0 27.0 48000.0]
 [0.0 1.0 0.0 30.0 54000.0]
 [0.0 0.0 1.0 38.0 61000.0]
 [0.0 1.0 0.0 40.0 63777.77777777778]
 [1.0 0.0 0.0 35.0 58000.0]
 [0.0 0.0 1.0 38.77777777777778 52000.0]
 [1.0 0.0 0.0 48.0 79000.0]
 [0.0 1.0 0.0 50.0 83000.0]
 [1.0 0.0 0.0 37.0 67000.0]]


### Encoding the Dependent Variable

In [ ]:
# APLICAR A VARIAVEIS CATEGORICAS QUANDO A ORDEM INTERESSA
#EXEMPLO - TAMANHOS DE ROUPA
#Para aplicar -> definir Y como a coluna à qual se quer aplicar. Podes ver o caso em baixo caso queiras que ele procure a coluna por nomenclatura
#ou, por exemplo -> coluna 3: X[:, 2] = le.fit_transform(X[:, 2])
#
#A variavel dependente é Yes ou No portanto é categórica
#Vamos fazer para ficar 0 ou 1 (No or Yes)
#Binary Outcome


from sklearn.preprocessing import LabelEncoder #encoder
le = LabelEncoder() #object. Não precisamos de dar inputs porque é só uma coluna
Y = le.fit_transform(Y)

#NOTA ALT: Caso quisesse selecionar uma coluna que ja sei que é a variavel dependente (e nao é a última)
#le = LabelEncoder()
#y = le.fit_transform(dataset['Survived'])

In [ ]:
print(Y)

[0 1 0 0 1 1 0 1 0 1]


## Splitting the dataset into the Training set and Test set

In [ ]:
#Treinar o modelo em existing data sets
#Avaliar o output do modelo correlacionando com data test
#X train, X test ; Y train, Y test. Quatro
from sklearn.model_selection import train_test_split #funcao para split de train e test

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2, random_state = 1)
#variaveis dependentes (X), variaveis independentes (Y), 20% das observacoes vao para o teste (escolhidas randomly)
#random_state = 1 fica fixed portanto nao fica random


In [ ]:
print(X_train) #8 observacoes pq metemos test size = 0.2 e temos 10 obs no total

[[0.0 0.0 1.0 38.77777777777778 52000.0]
 [0.0 1.0 0.0 40.0 63777.77777777778]
 [1.0 0.0 0.0 44.0 72000.0]
 [0.0 0.0 1.0 38.0 61000.0]
 [0.0 0.0 1.0 27.0 48000.0]
 [1.0 0.0 0.0 48.0 79000.0]
 [0.0 1.0 0.0 50.0 83000.0]
 [1.0 0.0 0.0 35.0 58000.0]]


In [ ]:
print(X_test)

[[0.0 1.0 0.0 30.0 54000.0]
 [1.0 0.0 0.0 37.0 67000.0]]


In [ ]:
print(Y_train) #nota importante - correspondem aos customers de cima (X_train)!

[0 1 0 0 1 1 0 1]


In [ ]:
print(Y_test)

[0 1]


## Feature Scaling

In [ ]:
#PARA OUTRAS APLICACOES -> PODE-SE APLICAR A TUDO. TIRAR AS COLUNAS AÍ. EX: ANN.

#Tem que ser depois de diferenciar o training set e o test set. Nem todos fazem assim
#Mas deve-se ao facto de:
# Test set tem que ser um novo brand new set (new observations). Como feature scaling implica mexer nos dados
# Prevents information leakage on the test set (which should not happen until the training is done).

#They must all take values in the same scale. No characteristic should override the others.

from sklearn.preprocessing import StandardScaler
sc = StandardScaler() #object do standard scaler
#standard scaler aplica-se mesmo aos valores dummy (array dos países, por exemplo)? Não. Não é preciso. Porque eles já estão entre 0 e 1

#lets fit into train set. We'll only take the columns of age and salary because country is an array one-hot encoded
#Olhando para cima, [0,0,1,38,52000] --> queremos a 3 e 4
X_train[:, 3:] = sc.fit_transform(X_train[:, 3:])
#O fit calcula media e standard deviation de cada coluna
#transform vai aplicar a formula.
#fit_transform faz os dois ao mesmo tempo

#Os scalers têm que ser iguais no training set e no test set
#Porquê? porque queremos a mesma aplicação entre dados.
X_test[:, 3:] = sc.fit_transform(X_test[:, 3:])

#NOTA IMP - na logistic regression tirámos a selecao das colunas [:, 3:] em ambos os lados. Provavelmente dá para fazer igual
#em todo o lado


In [ ]:
print(X_train)

[[0.0 0.0 1.0 -0.19159184384578545 -1.0781259408412425]
 [0.0 1.0 0.0 -0.014117293757057777 -0.07013167641635372]
 [1.0 0.0 0.0 0.566708506533324 0.633562432710455]
 [0.0 0.0 1.0 -0.30453019390224867 -0.30786617274297867]
 [0.0 0.0 1.0 -1.9018011447007988 -1.420463615551582]
 [1.0 0.0 0.0 1.1475343068237058 1.232653363453549]
 [0.0 1.0 0.0 1.4379472069688968 1.5749910381638885]
 [1.0 0.0 0.0 -0.7401495441200351 -0.5646194287757332]]


In [ ]:
print(X_test)

[[0.0 1.0 0.0 -1.0 -1.0]
 [1.0 0.0 0.0 1.0 1.0]]
